In [1]:
import mysql.connector
import numpy as np
import pandas as pd
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
import math
from sklearn.model_selection import  train_test_split

In [2]:
conn = mysql.connector.connect(
         user='foouser',
         password='F88Pa%%**',
         host='134.209.144.239',
         database='stocksdb') 

In [3]:
df=pd.read_sql("SHOW TABLES",con=conn)

In [4]:
df

,Tables_in_stocksdb
0,interview


In [5]:
df = pd.read_sql('SELECT * FROM interview', con=conn)

In [ ]:
df.isnull().sum()

In [6]:
new_dates, new_times = zip(*[(d.date(), d.time()) for d in df['datetime']])
df = df.assign(date=new_dates, time=new_times)

In [7]:
df.head()

,isin,insertion_datetime,datetime,open,high,low,close,volume,open_interest,date,time
0,INE545A01016,NaT,2018-01-01 09:15:00,2419.0,2436.00,2408.90,2435.25,10281,NaN,2018-01-01,09:15:00
1,INE545A01016,NaT,2018-01-01 09:16:00,2435.0,2440.00,2426.45,2439.00,13315,NaN,2018-01-01,09:16:00
2,INE545A01016,NaT,2018-01-01 09:17:00,2439.7,2439.70,2422.10,2424.95,3760,NaN,2018-01-01,09:17:00
3,INE545A01016,NaT,2018-01-01 09:18:00,2420.0,2424.50,2386.50,2395.00,11653,NaN,2018-01-01,09:18:00
4,INE545A01016,NaT,2018-01-01 09:19:00,2395.0,2413.95,2394.70,2409.85,7744,NaN,2018-01-01,09:19:00


In [8]:
df.drop(['datetime','insertion_datetime','open_interest','time','isin','date'],axis=1,inplace=True)

In [9]:
df.head()

,open,high,low,close,volume
0,2419.0,2436.00,2408.90,2435.25,10281
1,2435.0,2440.00,2426.45,2439.00,13315
2,2439.7,2439.70,2422.10,2424.95,3760
3,2420.0,2424.50,2386.50,2395.00,11653
4,2395.0,2413.95,2394.70,2409.85,7744


In [10]:
df = df[['open',  'high',  'low',  'close', 'volume']]
df['hl'] = (df['high'] - df['low']) / df['close'] * 100.0
df['co'] = (df['close'] - df['open']) / df['open'] * 100.0


In [11]:
df.head()

,open,high,low,close,volume,hl,co
0,2419.0,2436.00,2408.90,2435.25,10281,1.112822,0.671765
1,2435.0,2440.00,2426.45,2439.00,13315,0.555556,0.164271
2,2439.7,2439.70,2422.10,2424.95,3760,0.725788,-0.604583
3,2420.0,2424.50,2386.50,2395.00,11653,1.586639,-1.033058
4,2395.0,2413.95,2394.70,2409.85,7744,0.798805,0.620042


In [16]:
df = df[['close', 'hl', 'co', 'volume']]
target_col = 'close'


In [17]:
df.fillna(value=-99999, inplace=True)

In [18]:
target_out = int(math.ceil(0.01 * len(df)))
df['label'] = df[target_col].shift(-target_out)

In [19]:
X = np.array(df.drop(['label'], 1))
X = preprocessing.scale(X)
X = X[:-target_out]
df.dropna(inplace=True)
y = np.array(df['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [20]:
clf = LinearRegression(n_jobs=-1)
clf.fit(X_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [21]:
clf.fit

<bound method LinearRegression.fit of LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)>

In [22]:
confidence = clf.score(X_test, y_test)
print(confidence)

0.7756099721209979


In [23]:
clf.predict(X_test)

array([ 579.1420812 ,  849.10888634, 1615.39105569, ...,  745.30132472,
        169.40608108, 3288.79672088])